In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

# 템플릿 매칭 - 원본 이미지에서 다른 이미지와 동일한 이미지를 찾는 것

In [15]:
#필요한 이미지 로드
src = cv2.imread("./data/alphabet.bmp", cv2.IMREAD_GRAYSCALE)
tmp_A = cv2.imread("./data/A.bmp", cv2.IMREAD_GRAYSCALE)
tmp_S = cv2.imread("./data/S.bmp", cv2.IMREAD_GRAYSCALE)
tmp_b = cv2.imread("./data/b.bmp", cv2.IMREAD_GRAYSCALE)

#출력 영상
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

#찾기
result_A = cv2.matchTemplate(src, tmp_A, cv2.TM_SQDIFF_NORMED)
#찾은 결과의 최소값 과 위치를 찾음
minValue, _, minLocation, _ = cv2.minMaxLoc(result_A)
print(minValue, minLocation)

#찾은 영역에 사각형 그리기

#템플릿의 높이와 너비를 계산
w, h = tmp_A.shape[:2]
cv2.rectangle(dst, minLocation,
              (minLocation[0] + h, minLocation[1] + w), (255, 0 , 0), 3)

#원본 이미지 확인
cv2.imshow("dst", dst)





cv2.waitKey(0)
cv2.destroyAllWindows()


3.929205263375479e-07 (18, 20)


## 크기 변경

In [29]:
#필요한 이미지 로드
src = cv2.imread("./data/scaling.jpg", cv2.IMREAD_GRAYSCALE)
#원본 크기 확인
print(src.shape)

#반복문을 이용해서 이미지의 크기를 변경하는 방식 - 순방향 사상
def scaling_loop(img, size):
    start_time = time.perf_counter()
    
    dst = np.zeros(size[::-1], img.dtype)
    #확대 축소 비율 계산
    ratioY, ratioX = np.divide(size[::-1], img.shape[:2])
  
    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            i, j = int(y * ratioY), int(x * ratioX)
            dst[i, j] = img[y, x]
            
    elapsed = (time.perf_counter() - start_time) * 1000
    print("반복문 수행 시간:", "%0.2f ms" % elapsed)
    
    return dst

#행렬 단위 접근을 이용해서 이미지의 크기를 변경하는 방식
def scaling_matrix(img, size):
    start_time = time.perf_counter()
    
    dst = np.zeros(size[::-1], img.dtype)
    #확대 축소 비율 계산
    ratioY, ratioX = np.divide(size[::-1], img.shape[:2])
  
    y = np.arange(0, img.shape[0], 1)
    x = np.arange(0, img.shape[1], 1)
    
    #격자 그리드 생성
    y, x = np.meshgrid(y, x)
    i, j = np.int32(y * ratioY), np.int32(x * ratioX)
    dst[i,j] = img[y, x]
    
    elapsed = (time.perf_counter() - start_time) * 1000
    print("행렬이용 수행 시간:", "%0.2f ms" % elapsed)
    
    return dst

dst_loop = scaling_loop(src, (150, 125))
dst_matrix = scaling_matrix(src, (150, 125))
#원본 이미지 확인
cv2.imshow("src", src)

cv2.imshow("dst_loop", dst_loop)
cv2.imshow("dst_matrix", dst_matrix)

cv2.waitKey(0)
cv2.destroyAllWindows()

(300, 250)
반복문 수행 시간: 73.77 ms
행렬이용 수행 시간: 3.51 ms


## 보간

### 최근접 이웃 보간

In [32]:
#필요한 이미지 로드
src = cv2.imread("./data/scaling.jpg", cv2.IMREAD_GRAYSCALE)

#최근접 이웃 보간
def scaling_nearest(img, size):
       
    dst = np.zeros(size[::-1], img.dtype)
    #확대 축소 비율 계산
    ratioY, ratioX = np.divide(size[::-1], img.shape[:2])
  
    i = np.arange(0, size[1], 1)
    j = np.arange(0, size[0], 1)
    
    #격자 그리드 생성
    i, j = np.meshgrid(i, j)
    
    y, x = np.int32(i / ratioY), np.int32(j / ratioX)
    dst[i,j] = img[y, x]
    
    return dst

dst_nearest = scaling_nearest(src, (600, 500))

#원본 이미지 확인
cv2.imshow("src", src)
cv2.imshow("dst_nearest", dst_nearest)


cv2.waitKey(0)
cv2.destroyAllWindows()

### OpenCV 의 크기 변경 함수의 보간 적용

In [35]:
#필요한 이미지 로드
src = cv2.imread("./data/scaling.jpg", cv2.IMREAD_GRAYSCALE)

#최근접 이웃 보간을 이용한 확대
dst_nearest = cv2.resize(src, (600, 500), 0, 0, cv2.INTER_NEAREST)
#양선형 보간을 이용한 확대
dst_bilinear = cv2.resize(src, (600, 500), 0, 0, cv2.INTER_LINEAR)

#원본 이미지 확인
cv2.imshow("src", src)
cv2.imshow("dst_nearest", dst_nearest)
cv2.imshow("dst_bilinear", dst_bilinear)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Affine 변환 -  행렬을 이용해서 변환을 수행

In [39]:
#필요한 이미지 로드
src = cv2.imread("./data/affine.jpg", cv2.IMREAD_GRAYSCALE)

#affine 행렬을 만들기 위한 배열
pt1 = np.array([(30, 70), (20, 240), (300, 100)], np.float32)
pt2 = np.array([(120, 20), (10, 180), (280, 260)], np.float32) 

aff_mat = cv2.getAffineTransform(pt1, pt2)

#affine 행렬을 만들기 위한 데이터
center = (200, 200)
angle = 30
scale = 1

rot_mat = cv2.getRotationMatrix2D(center, angle, scale)


#affine 행렬을 이용한 변환
dst1 = cv2.warpAffine(src, aff_mat, src.shape[::-1], cv2.INTER_LINEAR)
dst2 = cv2.warpAffine(src, rot_mat, src.shape[::-1], cv2.INTER_LINEAR)

#원본 이미지 확인
cv2.imshow("src", src)
cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 원근 투시 변환

In [43]:
#필요한 이미지 로드
src = cv2.imread("./data/perspective.jpg", cv2.IMREAD_GRAYSCALE)

#원근을 적용할 4개의 점 좌표
pt1 = np.float32([(80, 40), (315, 133), (75, 300), (335, 300)])
#보정된 좌표
pt2 = np.float32([(50, 60), (340, 60), (50, 320), (340, 320)])

#원근 변환 행렬 생성
perspect_mat = cv2.getPerspectiveTransform(pt1, pt2)
print(perspect_mat)

dst = cv2.warpPerspective(src, perspect_mat, src.shape[1::-1])

#원본 이미지 확인
cv2.imshow("src", src)
cv2.imshow("dst", dst)


cv2.waitKey(0)
cv2.destroyAllWindows()

[[ 6.25789284e-01  3.98298577e-02 -6.88839366e+00]
 [-5.02676539e-01  1.06358288e+00  5.13923399e+01]
 [-1.57086418e-03  5.25700042e-04  1.00000000e+00]]


# 이미지 분할 - 관심 영역을 분리

## 에지 검출

### kenny 알고리즘

In [45]:
#필요한 이미지 로드
src = cv2.imread("./data/lena.jpg", cv2.IMREAD_GRAYSCALE)

dst1 = cv2.Canny(src, 50, 200)
dst2 = cv2.Canny(src, 50, 100)

#원본 이미지 확인
cv2.imshow("src", src)
cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [47]:
#필요한 이미지 로드
src = cv2.imread("./data/plane_256x256.jpg", cv2.IMREAD_GRAYSCALE)

#중간 픽셀에서 위아래 표준편차 1 정도를 적용
median_intensity = np.median(src)

lower_threshold = int(max(0, (1.0 - 0.33) * median_intensity))
upper_threshold = int(min(255, (1.0 + 0.33) * median_intensity))

dst1 = cv2.Canny(src, lower_threshold, upper_threshold)

#원본 이미지 확인
cv2.imshow("src", src)
cv2.imshow("dst1", dst1)

cv2.waitKey(0)
cv2.destroyAllWindows()